In [1]:
import os
print(os.getcwd())
def update_working_directory():
    from pathlib import Path
    p = Path(os.getcwd()).parents[0]
    os.chdir(p)
    print(p)
update_working_directory()

/Users/admin/Projects/vocabulary_learning/notebooks
/Users/admin/Projects/vocabulary_learning


In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime

In [4]:
def get_historical_data(historical_data_path='data/raw/historical_data.csv'):

    historical_data = pd.read_csv(historical_data_path)
    
    # historical_data = historical_data[~historical_data['id_vocab'].isna()]
    historical_data = historical_data[~historical_data['guess'].isna()]
    
    historical_data['language_asked'] = historical_data['language_asked'].str.lower()
    
    historical_data.loc[
        historical_data['day'].isna(), 'day'] = historical_data[
        historical_data['day'].isna()]['datetime'].apply(
        lambda x: datetime.datetime.strftime(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S.%f'),'%Y-%m-%d')
    )
    
    historical_data['day'] = historical_data['day'].map(
        lambda x: datetime.datetime.strptime(x, '%Y-%m-%d')
    )
    
    return historical_data

In [13]:
historical_data = get_historical_data()
historical_data['id_historical_data'] = range(len(historical_data))
historical_data

,english_word,german_word,language_asked,result,score_before,day,guess,datetime,id_vocab,id_historical_data
75,to see,sehen,german,1.0,-1.0,2020-09-07,sehen,2020-09-07 19:25:54.541437,NaN,0
76,fast,schnell,english,1.0,0.0,2020-09-07,fast,2020-09-07 19:25:57.364849,NaN,1
77,to study,studieren,german,1.0,0.0,2020-09-07,studieren,2020-09-07 19:26:01.860752,NaN,2
78,excuse-me,entschuldigung,german,1.0,0.0,2020-09-07,entschuldigung,2020-09-07 19:26:07.748328,NaN,3
79,the vegetables,das Gemüse,english,1.0,0.0,2020-09-07,the vegetables,2020-09-07 19:26:12.171932,NaN,4
...,...,...,...,...,...,...,...,...,...,...
747,see you later,bis Später,english,1.0,1.0,2020-09-20,see you later,2020-09-20 23:10:57.167500,263.0,644
748,the pants,die Hose,german,1.0,1.0,2020-09-20,die Hose,2020-09-20 23:11:00.775139,97.0,645
749,the grandpa,der Opa,english,1.0,1.0,2020-09-20,the grandpa,2020-09-20 23:11:06.207603,62.0,646
751,someone,jemand,german,1.0,0.0,2020-09-20,jemand,2020-09-20 23:11:24.150481,451.0,647


In [6]:
historical_data_test = historical_data[
    historical_data['english_word'].isin(['today','the mother','the oil', 'the egg'])
    | historical_data['german_word'].isin(['der Herbst','der Junge'])
].copy()
historical_data_test

,english_word,german_word,language_asked,result,score_before,day,guess,datetime,id_vocab,id_historical_data
80,today,heute,english,1.0,1.0,2020-09-07,today,2020-09-07 19:27:42.920885,NaN,5
81,the oil,das Öl,german,1.0,1.0,2020-09-07,das Öl,2020-09-07 19:27:46.400583,NaN,6
196,today,heute,german,1.0,2.0,2020-09-09,heute,2020-09-09 19:05:22.033941,148.0,116
202,the boy,der Junge,english,1.0,-2.0,2020-09-09,the boy,2020-09-09 19:05:52.192350,52.0,122
356,today,heute,german,1.0,2.0,2020-09-09,heute,2020-09-09 19:05:22.033941,148.0,269
362,the boy,der Junge,english,1.0,-2.0,2020-09-09,the boy,2020-09-09 19:05:52.192350,52.0,275
447,the egg,der Ei,english,1.0,-2.0,2020-09-09,the egg,2020-09-09 19:23:45.283346,45.0,357
451,the oil,das Öl,english,1.0,2.0,2020-09-09,the oil,2020-09-09 19:24:02.034865,29.0,361
529,the mother,die Mutter,english,1.0,0.0,2020-09-09,the mother,2020-09-09 20:42:53.375558,106.0,433
551,the autumn,der Herbst,english,1.0,0.0,2020-09-09,the autumn,2020-09-09 20:45:00.298046,49.0,454


In [7]:
# use dill - works the same way as pickle
import dill
with open('data/interim/historical_data_test.pkl', 'wb') as file:
    dill.dump(historical_data_test, file)

# Number of previous occurrences/successes/fails before

In [11]:
#To get something back
import pickle
with open("data/interim/historical_data_test.pkl", "rb") as input_file:
    historical_data_test = pickle.load(input_file)

In [7]:
def add_nb_previous_occurrences(historical_data):
    
    historical_data['occurrence'] = 1
    
    previous_occurrences = historical_data.groupby(
        ['english_word','id_historical_data']
    )['occurrence'].sum().groupby(level=0).cumsum().reset_index()
    previous_occurrences.rename(columns={
        'occurrence': 'previous_occurrences'
    }, inplace=True)
    previous_occurrences['previous_occurrences'] -= 1
    
    historical_data = pd.merge(
        historical_data,
        previous_occurrences[['id_historical_data', 'previous_occurrences']],
        on='id_historical_data'
    )
    
    del historical_data['occurrence']
    
    
    #score_before = success - fails
    #occurrence = success + fail

    #success = score_before + fails
    #success = ocurrence - fail
    #score_before + fails = ocurrence - fail
    #occurrence - score_before = 2 * fails

    #fails = success - score_before
    #fails = occurrence - success
    #success - score_before = occurrence - success
    #occurence + score_before = 2*success

    historical_data['previous_successes'] = (
        historical_data['previous_occurrences'] + historical_data['score_before']
    )/2
    historical_data['previous_fails'] = (
        historical_data['previous_occurrences'] - historical_data['score_before']
    )/2
    
    return historical_data
    

In [12]:
historical_data_test = add_nb_previous_occurrences(historical_data_test)
historical_data_test[
    historical_data_test['english_word'].isin(['today','the mother','the oil', 'the egg'])
    | historical_data_test['german_word'].isin(['der Herbst','der Junge'])
]

,english_word,german_word,language_asked,result,score_before,day,guess,datetime,id_historical_data,previous_occurrences,previous_successes,previous_fails
0,the mother,die Mutter,English,1,0,2020-07-02,NaN,NaN,0,0,0.0,0.0
1,the autumn,der Herbst,German,0,0,2020-07-02,NaN,NaN,4,0,0.0,0.0
2,the egg,der Ei,German,0,0,2020-07-02,NaN,NaN,16,0,0.0,0.0
3,the boy,der Junge,English,0,0,2020-07-02,NaN,NaN,19,0,0.0,0.0
4,the egg,der Ei,German,0,-1,2020-07-04,NaN,NaN,33,1,0.0,1.0
5,today,heute,English,1,0,2020-07-04,NaN,NaN,36,0,0.0,0.0
6,the oil,das Öl,German,1,0,2020-07-04,NaN,NaN,46,0,0.0,0.0
7,the boy,der Junge,german,0,-1,2020-09-07,NaN,NaN,74,1,0.0,1.0
8,today,heute,english,1,1,2020-09-07,today,2020-09-07 19:27:42.920885,80,1,1.0,0.0
9,the oil,das Öl,german,1,1,2020-09-07,das Öl,2020-09-07 19:27:46.400583,81,1,1.0,0.0


# Last occurrence

In [10]:
def add_last_occurrence(historical_data):
    
    # Calculate the difference between rows - By default, periods = 1
    historical_data['days_since_last_occurrences'] = historical_data.groupby('english_word')['day'].diff()
    
    historical_data['day_success'] = historical_data['day']
    historical_data.loc[historical_data['result'] != 1, 'day_success'] = None
    
    # Calculate the difference between rows - By default, periods = 1
    historical_data['days_since_last_success'] = historical_data.groupby('english_word')['day_success'].diff()
    
    del historical_data['day_success']
    
    return historical_data

In [11]:
historical_data = add_last_occurrence(historical_data)
historical_data

,english_word,german_word,language_asked,result,score_before,day,guess,datetime,id_historical_data,previous_occurrences,previous_successes,previous_fails,days_since_last_occurrences,days_since_last_success
0,the mother,die Mutter,English,1,0,2020-07-02,NaN,NaN,0,0,0.0,0.0,NaT,NaT
1,to run,rennen,English,1,0,2020-07-02,NaN,NaN,1,0,0.0,0.0,NaT,NaT
2,to believe,glauben,German,0,0,2020-07-02,NaN,NaN,2,0,0.0,0.0,NaT,NaT
3,the tomato,die Tomate,English,1,0,2020-07-02,NaN,NaN,3,0,0.0,0.0,NaT,NaT
4,the autumn,der Herbst,German,0,0,2020-07-02,NaN,NaN,4,0,0.0,0.0,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,today,heute,english,1,1,2020-09-07,today,2020-09-07 19:27:42.920885,80,1,1.0,0.0,65 days,65 days
81,the oil,das Öl,german,1,1,2020-09-07,das Öl,2020-09-07 19:27:46.400583,81,1,1.0,0.0,65 days,65 days
82,fantastic,fantastisch,english,1,0,2020-09-07,fantastic,2020-09-07 19:27:52.216245,82,0,0.0,0.0,NaT,NaT
83,big,gross,english,1,1,2020-09-07,big,2020-09-07 19:27:55.240736,83,1,1.0,0.0,65 days,65 days


# Error in guess

In [31]:
historical_data_test = historical_data[
    historical_data['english_word'].isin(['the vegetables','today', 'the student'])
    | historical_data['german_word'].isin([])
].copy()
historical_data_test

,english_word,german_word,language_asked,result,score_before,day,guess,datetime,id_vocab,id_historical_data,german_word_article,guess_article,correct_article
79,the vegetables,das Gemüse,english,1.0,0.0,2020-09-07,the vegetables,2020-09-07 19:26:12.171932,NaN,4,None,None,False
80,today,heute,english,1.0,1.0,2020-09-07,today,2020-09-07 19:27:42.920885,NaN,5,None,None,False
85,the student,der Student,german,1.0,0.0,2020-09-07,der Student,2020-09-07 22:14:17.794246,75.0,10,der,der,True
87,the vegetables,das Gemüse,german,0.0,1.0,2020-09-07,die Gemüse,2020-09-07 22:14:41.232593,23.0,12,das,die,False


## Error in article in German

In [58]:
def get_german_article(x, list_german_article = ['der','die','das']):
    possible_article = x.split(' ', 1)[0]
    if possible_article in list_german_article:
        return possible_article
    else:
        return None

In [64]:
def add_correct_article(historical_data, list_german_article = ['der','die','das']):
    
    historical_data['german_word_article'] = None
    historical_data.loc[
        historical_data['language_asked'] == 'german',
        "german_word_article"
    ] = historical_data.loc[
        historical_data['language_asked'] == 'german',
        "german_word"
    ].map(get_german_article)

    historical_data['guess_article'] = None
    historical_data.loc[
        historical_data['language_asked'] == 'german',
        "guess_article"
    ] = historical_data.loc[
        historical_data['language_asked'] == 'german',
        "guess"
    ].map(get_german_article)
    
    historical_data['correct_article'] = None
    historical_data.loc[
        (historical_data['language_asked'] == 'german')
        & (historical_data['german_word_article'].isin(list_german_article)),
        'correct_article'
    ] = historical_data['german_word_article'] == historical_data['guess_article']
    
    del historical_data['german_word_article']
    del historical_data['guess_article']

    return historical_data

In [65]:
historical_data = add_correct_article(historical_data)
historical_data

,english_word,german_word,language_asked,result,score_before,day,guess,datetime,id_vocab,id_historical_data,correct_article
75,to see,sehen,german,1.0,-1.0,2020-09-07,sehen,2020-09-07 19:25:54.541437,NaN,0,None
76,fast,schnell,english,1.0,0.0,2020-09-07,fast,2020-09-07 19:25:57.364849,NaN,1,None
77,to study,studieren,german,1.0,0.0,2020-09-07,studieren,2020-09-07 19:26:01.860752,NaN,2,None
78,excuse-me,entschuldigung,german,1.0,0.0,2020-09-07,entschuldigung,2020-09-07 19:26:07.748328,NaN,3,None
79,the vegetables,das Gemüse,english,1.0,0.0,2020-09-07,the vegetables,2020-09-07 19:26:12.171932,NaN,4,None
80,today,heute,english,1.0,1.0,2020-09-07,today,2020-09-07 19:27:42.920885,NaN,5,None
81,the oil,das Öl,german,1.0,1.0,2020-09-07,das Öl,2020-09-07 19:27:46.400583,NaN,6,True
82,fantastic,fantastisch,english,1.0,0.0,2020-09-07,fantastic,2020-09-07 19:27:52.216245,NaN,7,None
83,big,gross,english,1.0,1.0,2020-09-07,big,2020-09-07 19:27:55.240736,NaN,8,None
84,I agree,genau,german,1.0,0.0,2020-09-07,genau,2020-09-07 19:27:59.344412,NaN,9,None


## Levenshtein difference with guess

In [36]:
from Levenshtein import distance
list_german_article = ['der','die','das']
list_english_article = ['the','to']

In [48]:
# Lowercase
historical_data_test['german_word_lv'] = historical_data_test['german_word'].str.lower()
historical_data_test['english_word_lv'] = historical_data_test['english_word'].str.lower()
historical_data_test['guess_lv'] = historical_data_test['guess'].str.lower()

In [49]:
historical_data_test['german_word_lv'] = historical_data_test['german_word_lv'].map(
    lambda x: ' '.join(word for word in x.split(' ') if word not in list_german_article)
)
historical_data_test['english_word_lv'] = historical_data_test['english_word_lv'].map(
    lambda x: ' '.join(word for word in x.split(' ') if word not in list_english_article)
)

historical_data_test.loc[
    historical_data_test['language_asked'] == 'german',
    'guess_lv'
] = historical_data_test['guess_lv'].map(
    lambda x: ' '.join(word for word in x.split(' ') if word not in list_german_article)
)
historical_data_test.loc[
    historical_data_test['language_asked'] == 'english',
    'guess_lv'
] = historical_data_test['guess_lv'].map(
    lambda x: ' '.join(word for word in x.split(' ') if word not in list_english_article)
)

In [53]:
historical_data_test['levenshtein_dist'] = None
historical_data_test.loc[
    historical_data_test['language_asked'] == 'german',
    "levenshtein_dist"
] = historical_data_test.apply(lambda x: distance(x['german_word_lv'], x['guess_lv']), axis=1)
historical_data_test.loc[
    historical_data_test['language_asked'] == 'english',
    "levenshtein_dist"
] = historical_data_test.apply(lambda x: distance(x['english_word_lv'], x['guess_lv']), axis=1)

In [55]:
del historical_data_test['german_word_lv']
del historical_data_test['english_word_lv']
del historical_data_test['guess_lv']

In [56]:
historical_data_test

,english_word,german_word,language_asked,result,score_before,day,guess,datetime,id_vocab,id_historical_data,german_word_article,guess_article,correct_article,levenshtein_dist
79,the vegetables,das Gemüse,english,1.0,0.0,2020-09-07,the vegetables,2020-09-07 19:26:12.171932,NaN,4,None,None,False,0
80,today,heute,english,1.0,1.0,2020-09-07,today,2020-09-07 19:27:42.920885,NaN,5,None,None,False,0
85,the student,der Student,german,1.0,0.0,2020-09-07,der Student,2020-09-07 22:14:17.794246,75.0,10,der,der,True,0
87,the vegetables,das Gemüse,german,0.0,1.0,2020-09-07,die Gemüse,2020-09-07 22:14:41.232593,23.0,12,das,die,False,0


# Final

In [11]:
historical_data

,english_word,german_word,language_asked,result,score_before,day,guess,datetime,id_vocab,id_historical_data
0,the mother,die Mutter,English,1.0,0.0,2020-07-02,NaN,NaN,NaN,0
1,to run,rennen,English,1.0,0.0,2020-07-02,NaN,NaN,NaN,1
2,to believe,glauben,German,0.0,0.0,2020-07-02,NaN,NaN,NaN,2
3,the tomato,die Tomate,English,1.0,0.0,2020-07-02,NaN,NaN,NaN,3
4,the autumn,der Herbst,German,0.0,0.0,2020-07-02,NaN,NaN,NaN,4
...,...,...,...,...,...,...,...,...,...,...
85,the student,der Student,german,1.0,0.0,2020-09-07,der Student,2020-09-07 22:14:17.794246,75.0,85
86,fine,prima,english,0.0,0.0,2020-09-07,first,2020-09-07 22:14:30.921036,178.0,86
87,the vegetables,das Gemüse,german,0.0,1.0,2020-09-07,die Gemüse,2020-09-07 22:14:41.232593,23.0,87
88,English,Englisch,german,0.0,0.0,2020-09-07,english,2020-09-07 22:14:49.504302,121.0,88
